<a href="https://colab.research.google.com/github/kridtapon/KST-RVI-Momentum-Strategy/blob/main/KST_RVI_Momentum_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate the KST (Know Sure Thing) and Signal line
def calculate_kst(df, short_period=10, long_period=15, roc_short_period=5, roc_long_period=10):
    """
    Calculate KST (Know Sure Thing) and its Signal line.
    """
    # Rate of Change (ROC) for short and long periods
    roc_short = df['Close'].pct_change(periods=short_period) * 100
    roc_long = df['Close'].pct_change(periods=long_period) * 100
    roc_mid = df['Close'].pct_change(periods=roc_short_period) * 100
    roc_longer = df['Close'].pct_change(periods=roc_long_period) * 100

    # Smoothed ROC using weighted moving averages
    wma_short = roc_short.rolling(window=short_period).mean()
    wma_long = roc_long.rolling(window=long_period).mean()
    wma_mid = roc_mid.rolling(window=roc_short_period).mean()
    wma_longer = roc_longer.rolling(window=roc_long_period).mean()

    # KST = weighted sum of ROCs
    kst = (wma_short * 1) + (wma_long * 2) + (wma_mid * 3) + (wma_longer * 4)

    # Signal line: EMA of KST
    kst_signal = kst.ewm(span=9).mean()

    return kst, kst_signal

# Function to calculate KST for a specific time period
def calculate_kst_indicator(df, kst_params):
    kst, kst_signal = calculate_kst(df, *kst_params)
    df['KST'] = kst
    df['KST_Signal'] = kst_signal
    return df

# Function to calculate Relative Vigor Index (RVI)
def calculate_rvi(df, period=14):
    """
    Calculate Relative Vigor Index (RVI).
    """
    close_open = df['Close'] - df['Open']
    high_low = df['High'] - df['Low']

    rvi = close_open.rolling(window=period).sum() / high_low.rolling(window=period).sum()

    return rvi

# Define the stock symbol and time period
symbol = 'META'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Define parameters for KST calculation
kst_params = (10, 15, 5, 10)

# Calculate KST and Signal line
df = calculate_kst_indicator(df, kst_params)

# Calculate Relative Vigor Index (RVI)
df['RVI'] = calculate_rvi(df)

# Define Entry and Exit signals based on KST and RVI
df['Entry'] = (df['KST'] > df['KST_Signal']) & (df['RVI'] > 0)
df['Exit'] = (df['KST'] < df['KST_Signal']) & (df['RVI'] < 0)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           157970.712448
Total Return [%]                        57.970712
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                        7425.87823
Max Drawdown [%]                         56.44003
Max Drawdown Duration                       863.0
Total Trades                                   34
Total Closed Trades                            34
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            44.117647
Best Trade [%]                          46.999486
Worst Trade [%]                        -32.620541
Avg Winning Trade [%]                    12.24081
Avg Losing Trade [%]                    -5.596265
